In [1]:
# Imports and Setup
import os
import re
import json
import requests
import pandas as pd
from typing import List, Dict, Any, Optional, Union
from dataclasses import dataclass
from enum import Enum
import textwrap

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, AIMessage
from langchain_text_splitters import CharacterTextSplitter

from langchain.tools import tool

# LLM imports
from llama_index.llms.ollama import Ollama

import json
import urllib.request
import urllib.error
import urllib.parse
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

from bs4 import BeautifulSoup
from ddgs import DDGS

In [2]:
# Configuration and LLM Setup
REQUEST_TIMEOUT = 180
MODEL_NAME = "qwen2.5:14b"

# Initialize Ollama LLM
llm = Ollama(
    model=MODEL_NAME, 
    request_timeout=REQUEST_TIMEOUT
)

# Test LLM connection
test_response = llm.complete("Hello")
print(f"LLM initialized: {test_response.text[:50]}...")


def pprint(text):
    wrapped_lines = textwrap.wrap(text, width=130)
    for line in wrapped_lines:
        print(line)

LLM initialized: Hello! How can I assist you today? Feel free to as...


In [3]:
def create_plan(task: str) -> str:
    """
    Given a task, determine a step-by-step action plan of what needs to be done to accomplish this task and output the answer/result. 
    The most important actions that are taken: 
     1. Define the goal: what result is asked to be produced.
     2. List the steps: provide a short explanation for each action that needs to be taken.       
    """
    
    prompt = f"""
You are an expert project planner. Your task is to create a concise, step-by-step action plan to accomplish the user's goal.

User's Goal:
---
{task}
---

Instructions:
1. Clarify the Core Objective: Start by rephrasing the user's goal as a single, clear, and specific objective.
2. Develop a Chronological Action Plan: Break down the objective into a logical sequence of high-level steps.

Guiding Principles for the Plan:
- Tool-Agnostic: Focus on the action required, not the specific tool to perform it (e.g., use "Gather data on market trends" instead of "Search Google for market trends").
- Information First: The initial step should almost always be to gather and analyze the necessary information before taking further action.
- S.M.A.R. Steps: Each step must be Specific, Measurable, Achievable, and Relevant. The focus is on the logical sequence, not specific deadlines.
- Concise: Include only the critical steps needed to reach the objective.

Example Output Format (ALWAS **JSON** ):
{{
  "objective": "Plan and execute a one-day offsite event for a team of 10 people focused on team building and strategic planning.",
  "plan": [
    "Gather requirements including budget, potential dates, and key goals for the offsite from team leadership",
    "Research and shortlist suitable venues and activity options that fit the budget and goals",
    "Create a detailed agenda and budget proposal for approval",
    "Book the selected venue, catering, and activities upon approval",
    "Send out official invitations and manage attendee confirmations and dietary requirements",
    "Finalize all logistical details and communicate the full itinerary to the team"
  ]
}}
where
  "objective" 's value in the json is a clear, one-sentence summary of the end goal,
  "plan" 's value in the json is a list **ALWAYS SEPARATED BY PYTHON NEWLINE CHARCTER** like 
  [
    A short explanation of the first logical step", 
    A short explanation of the next step that follows from the first",
    And so on..."
  ]
"""
    task_response = llm.complete(prompt)

    return task_response.text

@dataclass
class Task:
    question: str
    objective: str
    plan: list[str]


def define_task(task:str) -> str: 
    task_plan_response = create_plan(task)

    # Assume llm outputs smth json-like with the correct keys.
    result = json.loads(task_plan_response)

    return Task(
      question=task,
      objective=result["objective"],
      plan=result["plan"],
    )

In [4]:
import json
from dataclasses import dataclass
from typing import List, Dict, Literal

# Define the available capabilities as a type for better validation
CapabilityAgents = Literal[
    "deep_web_search",
    "video_processing",
    "audio_processing",
    "image_processing",
    "structured_data_processing",
    "unstructured_data_processing",
    "code_math_writing"
]

@dataclass
class CapabilityPlan:
    """A structured plan outlining the sequence of capabilities and actions."""
    subplan: List[Dict[str, CapabilityAgents | str]]

def create_capability_plan(task: str, attachments: List[str] = None) -> CapabilityPlan:
    """
    Analyzes a task and generates a sequential execution plan using available capabilities.

    Args:
        task (str): The description of the task to be performed.
        attachments (list[str]): A list of file names related to the task.

    Returns:
        CapabilityPlan: A dataclass containing the ordered list of sub-tasks.
    """
    attachment_info = ""
    if attachments:
        attachment_info = f"\n\nAttachments provided: {', '.join(attachments)}"

    planning_prompt = f"""
You are a highly intelligent planning agent. Your primary function is to analyze a user's task and create a precise, step-by-step execution plan using a predefined set of capabilities.

**Your Task:**
Analyze the provided task and create a sequential plan to accomplish it. The plan should be a list of steps, where each step defines the capability to use and the specific activity to perform.

**Capabilities:**
- `deep_web_search`: Find, evaluate, and download web content (e.g., articles, documents). This capability is for search and downloading web resources only, not for processing the content or getting any answers on the content.
- `video_processing`: Download video, extract frames or audio from a video file for further analysis.
- `audio_processing`: Download audio, transcribe speech, identify sounds, or analyze properties of an audio file.
- `image_processing`: Download image, analyze an image to identify objects, read text (OCR), or understand its content.
- `structured_data_processing`: Analyze, query, or visualize data from structured files like Parquet, CSV, JSON, or databases.
- `unstructured_data_processing`: Analyze, summarize, extract information from, or answer questions about raw text or documents (e.g., PDFs, TXT files, retrieved web content).
- `code_math_writing`: Generate or execute code, solve mathematical problems, or perform complex logical operations and computations.

Instructions:
Deconstruct the Task -> Assign Capabilities for each step -> Define the Activity for each step (i.e.,write a clear and concise description of the specific action to be performed using the chosen capability)

Example 1: Simple Fact Lookup
Task: "What is the boiling point of water at sea level?"
Output:
{{
  "subplan": [
    {{
      "capability": "deep_web_search",
      "activity": "Search for the boiling point of water at sea level"
    }},
    {{
      "capability": "unstructured_data_processing",
      "activity": "Analyze the downloaded web resources and find the reference to the boiling point temperature."
    }}
  ]
}}

Example 2: Multi-step Information Retrieval and Analysis

Task: "Find the Q2 2025 earnings report for NVIDIA and tell me what their 'Gaming' division revenue was."
Output:
{{
  "subplan": [
    {{
      "capability": "deep_web_search",
      "activity": "Search for and download NVIDIA's official Q2 2025 earnings report document and download it."
    }},
    {{
      "capability": "unstructured_data_processing",
      "activity": "Analyze the downloaded earnings report to find and extract the revenue figure for the 'Gaming' division."
    }}
  ]
}}

---
Begin Plan Generation

Task: "{task}"
Attachments: "{attachment_info}"

Respond in this exact JSON format:
{{
  "subplan": [
    {{
      "capability": "...",
      "activity": "..."
    }},
    {{
      "capability": "...",
      "activity": "..."
    }}
  ]
}}
"""
    response = llm.complete(planning_prompt)
    response_text = response.text.strip()

    result = json.loads(response_text)
    return CapabilityPlan(subplan=result["subplan"])

In [5]:
question = "Who did the actor who played Ray in the Polish-language version of Everybody Loves Raymond play in Magda M."

task = define_task(question)
pprint(task.__str__())

Task(question='Who did the actor who played Ray in the Polish-language version of Everybody Loves Raymond play in Magda M.',
objective="Identify the actor who played Ray in the Polish-language version of 'Everybody Loves Raymond' and determine their role
in the movie 'Magda M.'", plan=["Research to identify the title or details of the Polish-language adaptation of 'Everybody Loves
Raymond'", 'Find the name of the actor who played Ray in this adaptation', "Gather information on the filmography of the
identified actor, focusing on their role in 'Magda M.'"])


In [6]:
# Iterate over the plan
# Get a single task of the plan together with the objective 
current_plan_step = task.plan[0]
task_step = f"You need to FOCUS ON: {current_plan_step}. When your global objective which you SHOULD NOT FOCUS ON: {task.objective}."  

# What is required for this to be executed?
capabilities = create_capability_plan(task=task_step)
pprint(capabilities.__str__())

CapabilityPlan(subplan=[{'capability': 'deep_web_search', 'activity': "Search for the Polish-language adaptation of 'Everybody
Loves Raymond' and download relevant web pages or documents."}, {'capability': 'unstructured_data_processing', 'activity':
"Analyze the downloaded content to extract information about the title or production details of the Polish-language version of
'Everybody Loves Raymond'."}])


In [7]:
##################
# web_search_agent 
##################

LINK_KEY = "link"

@dataclass
class WebResource:
    """A unified dataclass for handling a web resource.

    Attributes:
        content: The main text content of the web page. Can be None if not yet downloaded.
        link: The unique URL for the resource.
        metadata: A dictionary containing additional information, such as search result data.
    """
    content: Optional[str]
    link: str
    metadata: Dict[str, Any]


def question_to_queries(question: str, max_queries: int = 2) -> List[str]:
    """Converts a user question into a list of optimized search engine queries.

    Note:
        This function requires a Large Language Model (LLM) to generate queries.
        The `llm.complete()` call is a placeholder for your model's inference logic.

    Args:
        question: The user's input question.
        max_queries: The maximum number of search queries to generate.

    Returns:
        A list of string queries optimized for a search engine.
    """
    prompt = f"""
    Create a list of general search engine queries for the following question: "{question}".

    Make sure that:
    - Your output is a list separated by a "|" character and nothing else.
    - You provide a MAXIMUM of {max_queries} search engine queries.
    - Each query is SHORT and precise.

    Example Output:
    Large urban population areas in Europe|Biggest cities in Europe
    """
    llm_response = llm.complete(prompt)
    response_text = llm_response.text

    return response_text.strip().split("|")


def duckduckgo_search(query: str, max_results: int = 2) -> List[WebResource]:
    """Performs a DuckDuckGo search and returns results as WebResource objects.

    Args:
        query: The search query string.
        max_results: The maximum number of search results to retrieve.

    Returns:
        A list of WebResource objects, where 'content' is None and 'metadata'
        contains the search result details.
    """
    found_resources = []
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))
        if not results:
            print(f"⚠️ No search results found for '{query}'")
            return []

        for i, result in enumerate(results):
            resource = WebResource(
                content=None,
                link=result.get('href', 'No URL'),
                metadata={
                    "search_order": i,
                    "web_page_title": result.get('title', 'No title'),
                    "web_page_summary": result.get('body', 'No description'),
                    "query": query
                }
            )
            found_resources.append(resource)

    return found_resources


def drop_non_unique_resources(resources: List[WebResource]) -> List[WebResource]:
    """Removes duplicate WebResource objects based on their 'link' attribute.

    Args:
        resources: A list of WebResource objects.

    Returns:
        A new list of WebResource objects with duplicates removed.
    """
    seen_links = set()
    unique_resources = []
    for resource in resources:
        if resource.link and resource.link not in seen_links:
            unique_resources.append(resource)
            seen_links.add(resource.link)
    return unique_resources


def extract_clean_text(raw_html: str) -> str:
    """Extracts clean, readable text from raw HTML content.

    This function removes scripts, styles, navigation, and other non-content
    elements, then cleans up whitespace.

    Args:
        raw_html: The raw HTML content of a webpage.

    Returns:
        The extracted and cleaned plain text.
    """
    try:
        soup = BeautifulSoup(raw_html, 'html.parser')
        # Remove elements that typically do not contain main content
        for element in soup(["script", "style", "nav", "footer", "header", "aside"]):
            element.decompose()

        # Extract text and clean up whitespace
        text = soup.get_text(separator=" ")
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        return ' '.join(chunk for chunk in chunks if chunk)
    except Exception as e:
        print(f"Error extracting text from HTML: {e}")
        return ""


def download_content(resource: WebResource) -> WebResource:
    """Downloads the HTML from a resource's link and populates its 'content' field.

    Args:
        resource: A WebResource object with a valid 'link'.

    Returns:
        The same WebResource object with its 'content' field updated with
        the cleaned text from the webpage. Returns the original object if
        download fails.
    """
    if not resource.link:
        return resource

    try:
        with urllib.request.urlopen(resource.link, timeout=15) as response:
            html_content = response.read()
            resource.content = extract_clean_text(html_content)
    except urllib.error.URLError as e:
        print(f"URL Error for {resource.link}: {e}")
    except Exception as e:
        print(f"Unexpected error for {resource.link}: {e}")

    return resource


def web_search(question: str, links_per_query: int = 2) -> List[WebResource]:
    """Orchestrates the full web search process for a given question.

    This process includes:
    1. Converting the question into search queries.
    2. Searching the web to find resources.
    3. Downloading and extracting text content from each resource.

    Args:
        question: The user's question.
        links_per_query: The number of web links to retrieve for each search query.

    Returns:
        A list of WebResource objects, with their 'content' field populated.
    """
    # 1. Generate search queries from the question
    candidate_queries = question_to_queries(question)
    print(f"\nGenerated queries: {candidate_queries}")

    # 2. Search for relevant sources for each query
    all_sources = []
    for query in candidate_queries:
        sources_for_query = duckduckgo_search(query, links_per_query)
        all_sources.extend(sources_for_query)

    # 3. Filter out any duplicate resources found by different queries
    unique_sources = drop_non_unique_resources(all_sources)
    print(f"\nFound {len(unique_sources)} unique web resources.")

    # 4. Download content for each unique resource
    final_resources = []
    for source in unique_sources:
        populated_resource = download_content(source)
        if populated_resource.content: # Only keep resources where content was successfully downloaded
            final_resources.append(populated_resource)

    return final_resources

In [8]:
# Based on capabilities let an agent do the task 



# Combine task and objective and hand over the execution to the agent 

In [9]:
# Agent's work flow:
#  1. Compose query 
#  2. Do search & collect results 
#  3. Download and process results
#  4. Save results 
#  5. Produce summery of how the data looks like and where to find it.
 

In [10]:
#  Define agents 
from llama_index.core.agent import ReActAgent, FunctionCallingAgent
from llama_index.core.tools import FunctionTool